# Neural Network Asset Pricing: Echo State Network vs. Linear Regression

**Course:** Advanced Methods of Risk Management — University of Bologna, MSc Quantitative Finance (2024)

This notebook compares an Echo State Network (ESN) against a linear regression baseline for short-term stock price forecasting using Apple (AAPL) historical daily closing prices.

**Approach:**
- Full AAPL historical daily close prices via Alpha Vantage API
- 30-day lookback window to predict next-day close
- Min-Max scaling preprocessing
- 80/20 train/test split (temporal order preserved)
- Hyperparameter tuning via GridSearchCV

## 1. Imports

In [ ]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

## 2. Data Fetching & Preprocessing

In [ ]:
# Replace with your free Alpha Vantage API key: https://www.alphavantage.co
api_key = 'YOUR_API_KEY'

def fetch_stock_data(symbol):
    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_daily(symbol=symbol, outputsize='full')
    return data['4. close']

stock_symbol = 'AAPL'
stock_data = fetch_stock_data(stock_symbol)

# Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(np.array(stock_data).reshape(-1, 1))

print(f'Total data points: {len(scaled_data)}')

## 3. Sequence Creation & Train/Test Split

In [ ]:
def create_sequences(data, seq_length):
    """Create input/target sequences using a rolling window."""
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length, 0])
        y.append(data[i + seq_length, 0])
    return np.array(X), np.array(y)

seq_length = 30  # 30-day lookback window
X, y = create_sequences(scaled_data, seq_length)

# Temporal split — no shuffle to preserve time order
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=False
)

print(f'Training samples: {len(X_train)}, Test samples: {len(X_test)}')

## 4. Echo State Network (ESN) — Baseline

The ESN is approximated using a Polynomial Features + Ridge regression pipeline, capturing non-linear reservoir-like transformations of the input.

In [ ]:
# Baseline ESN (degree=2, default Ridge alpha)
esn_model = make_pipeline(PolynomialFeatures(2), Ridge())
esn_model.fit(X_train, y_train)

y_pred_esn = esn_model.predict(X_test)

mse_esn = mean_squared_error(y_test, y_pred_esn)
rmse_esn = np.sqrt(mse_esn)
mae_esn = mean_absolute_error(y_test, y_pred_esn)
r2_esn = r2_score(y_test, y_pred_esn)

print('Echo State Network (baseline):')
print(f'  MSE:  {mse_esn:.6e}')
print(f'  RMSE: {rmse_esn:.6e}')
print(f'  MAE:  {mae_esn:.6e}')
print(f'  R²:   {r2_esn:.6f}')

## 5. Linear Regression — Baseline

In [ ]:
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

y_pred_linear = linear_model.predict(X_test)

mse_linear = mean_squared_error(y_test, y_pred_linear)
rmse_linear = np.sqrt(mse_linear)
mae_linear = mean_absolute_error(y_test, y_pred_linear)
r2_linear = r2_score(y_test, y_pred_linear)

print('Linear Regression:')
print(f'  MSE:  {mse_linear:.6e}')
print(f'  RMSE: {rmse_linear:.6e}')
print(f'  MAE:  {mae_linear:.6e}')
print(f'  R²:   {r2_linear:.6f}')

## 6. ESN Hyperparameter Tuning — GridSearchCV

In [ ]:
param_grid = {
    'polynomialfeatures__degree': [1, 2, 3],
    'ridge__alpha': [0.001, 0.01, 0.1, 1]
}

esn_tuned = GridSearchCV(
    estimator=make_pipeline(PolynomialFeatures(), Ridge()),
    param_grid=param_grid,
    cv=5,
    scoring='neg_mean_squared_error'
)
esn_tuned.fit(X_train, y_train)

y_pred_tuned = esn_tuned.predict(X_test)

mse_tuned = mean_squared_error(y_test, y_pred_tuned)
rmse_tuned = np.sqrt(mse_tuned)
mae_tuned = mean_absolute_error(y_test, y_pred_tuned)
r2_tuned = r2_score(y_test, y_pred_tuned)

print('ESN (tuned):')
print(f'  Best params: {esn_tuned.best_params_}')
print(f'  MSE:  {mse_tuned:.6e}')
print(f'  RMSE: {rmse_tuned:.6e}')
print(f'  MAE:  {mae_tuned:.6e}')
print(f'  R²:   {r2_tuned:.6f}')

## 7. Results Summary

In [ ]:
results = pd.DataFrame({
    'Model': ['ESN (baseline)', 'Linear Regression', 'ESN (tuned)'],
    'MSE':  [mse_esn,    mse_linear,    mse_tuned],
    'RMSE': [rmse_esn,   rmse_linear,   rmse_tuned],
    'MAE':  [mae_esn,    mae_linear,    mae_tuned],
    'R²':   [r2_esn,     r2_linear,     r2_tuned]
})
print(results.to_string(index=False))

## 8. Visualisation

In [ ]:
import os
os.makedirs('plots', exist_ok=True)

# Baseline comparison
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Actual Values', linewidth=1.5)
plt.plot(y_pred_linear, label='Linear Regression', linewidth=1)
plt.plot(y_pred_esn, label='ESN (baseline)', linewidth=1)
plt.xlabel('Time')
plt.ylabel('Closing Price (scaled)')
plt.title('Actual vs Predicted Values — Baseline Models')
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig('plots/actual_vs_predicted_baseline.png', dpi=150, bbox_inches='tight')
plt.show()

# Tuned ESN comparison
plt.figure(figsize=(12, 6))
plt.plot(y_test, label='Actual Values', linewidth=1.5)
plt.plot(y_pred_linear, label='Linear Regression', linewidth=1)
plt.plot(y_pred_tuned, label='ESN (tuned)', linewidth=1)
plt.xlabel('Time')
plt.ylabel('Closing Price (scaled)')
plt.title('Actual vs Predicted Values — Tuned ESN')
plt.legend()
plt.grid()
plt.tight_layout()
plt.savefig('plots/actual_vs_predicted_tuned.png', dpi=150, bbox_inches='tight')
plt.show()

print('Plots saved to /plots')